In [30]:
import numpy as np 
import matplotlib.pyplot as plt 
from numpy.linalg import det 
%matplotlib osx 

In [31]:
def strat_rand(matrix,last_move):
    index = np.where(matrix == '-')
    #print(index)
    l = len(index[0])
    #print(l)
    z = rng.integers(0, l)
    i = index[0][z]
    j = index[1][z]
    return i,j 


In [32]:
def strat_next_to(matrix, last_move):
    n = np.shape(matrix)[0]
    a,b = last_move
    moves = [matrix[a,(b+1)%n],matrix[a,(b-1)%n], matrix[(a+1)%n,b], matrix[(a-1)%n,b]]
    index = np.nonzero(moves == '-')
    l = len(index[0])
    #print(index[0])
    if l == 0:
        i,j = strat_rand(matrix, last_move)
    else:
        z = rng.integers(0,l)
        if z == 0:
            i = a
            j = (b+1)%n
        elif z == 1:
            i = a
            j = (b-1)%n
        elif z == 2:
            i = (a+1)%n
            j = b
        elif z == 3:
            i = (a-1)%n
            j = b
    return i,j
    

In [33]:
def strat_diag_to(matrix, last_move):
    n = np.shape(matrix)[0]
    a,b = last_move
    #print(a)
    #print(b)
    moves = [matrix[(a+1)%n,(b+1)%n],matrix[(a+1)%n,(b-1)%n], matrix[(a-1)%n,(b+1)%n], matrix[(a-1)%n,(b-1)%n]]
    #print(moves)
    index = np.nonzero(moves == '-')
    l = len(index[0])
    #print(index)
    if l == 0:
        i,j = strat_rand(matrix, last_move)
    else:
        z = rng.integers(0,l)
        if z == 0:
            i = (a+1)%n
            j = (b+1)%n
        elif z == 1:
            i = (a+1)%n
            j = (b-1)%n
        elif z == 2:
            i = (a-1)%n
            j = (b+1)%n
        elif z == 3:
            i = (a-1)%n
            j = (b-1)%n
    return i,j

In [34]:
def strat_zero_rc(matrix,last_move):
    n = np.shape(matrix)[0]
    move_matrix = np.copy(matrix)
    #print(move_matrix)
    index = np.where(matrix == '1')
    for k in range(len(index[0])):
        a,b = index[0][k], index[1][k]
        for l in range(n):
            move_matrix[(a+l)%n, b] = 2
            move_matrix[(a, (b+l)%n)] = 2
    moves = np.where(move_matrix == '-')
    t = len(moves[0])
    if t == 0:
        i,j = strat_rand(matrix, last_move)
    else:
        z = rng.integers(0,t)
        i = moves[0][z]
        j = moves[1][z]
    return i,j

In [35]:
def strat_diagonals(matrix,last_move):
    n = np.shape(matrix)[0]
    move_matrix = np.zeros_like(matrix)
    weight = 0
    for j in range(n):
        pos = True
        count1 = 0
        neg = True 
        count2 = 0
        for k in range(n):
            #matrix[k,(j+k)%n]
            #matrix[k,(j-k)%n]
            if (matrix[k,(j+k)%n] == 0):
                pos = False
            elif (matrix[k,(j+k)%n] == 1):
                count1 += 1
                move_matrix[k,(j+k)%n] = -1
            if (matrix[k,(j-k)%n] == 0):
                neg = False
            elif (matrix[k,(j-k)%n] == 1):
                count2 += 1
                move_matrix[k,(j-k)%n] = -1
        if pos:
            for l in range(n):
                if not (move_matrix[k,(j+k)%n] == -1):
                    move_matrix[l,(j+l)%n] = count1
        if neg:
            for m in range(n):
                if not (move_matrix[k,(j-k)%n] == -1):
                    move_matrix[m,(j-m)%n] = count2
        c = np.max(count1, count2)
        if c > weight:
            weight = c
    #print('*'*100)
    #print(move_matrix)
    #print('*'*100)
    if weight > 0:
        index = np.where(move_matrix == weight)
        t = len(index[0])
        z = rng.integers(0,t)
        i = index[0][z]
        j = index[1][z]
    else:
        #Reconstruct move matrix to find a new strategy
        i,j = strat_rand(matrix, last_move)
    return i,j

In [36]:
def matrix_game_strategies(n, strat_player_0, strat_player_1, go = 1):
    matrix = np.full((n,n), '-')
    if go != 1 and go != 0:
        raise ValueError('The varibale go must be either 0 or 1')
    #print(matrix)
    #print()
    #print('-'*100)
    #print()
    last_move = (0,0) #Placement of first move is irrelevant
    #move_matrix = np.ones_like(matrix, dtype=int)
    while np.any(matrix == '-'):
        if go == 1:
            x,y = strat_player_0(matrix, last_move)
            matrix[x,y] = int(0)
            last_move = (x,y)
            go = 0
            #print(matrix)
            #print()
            #print('-'*100)
            #print()
        else:
            a,b = strat_player_1(matrix, last_move)
            matrix[a,b] = int(1)
            last_move = (a,b)
            go = 1
            #print(matrix)
            #print()
            #print('-'*100)
            #print()
    output = matrix.astype(int)
    d = det(output)
    #print(det)
    if d == 0:
        return 1
    else:
       return 0

In [48]:
sims = 1e3
n = 4
rng = np.random.default_rng()
count1 = 0
count2 = 0
count3 = 0
count4 = 0 
count5 = 0
count6 = 0
count7 = 0
count8 = 0
count9 = 0
for i in range(int(sims)):
    count1 += matrix_game_strategies(n, strat_rand, strat_rand)
    count2 += matrix_game_strategies(n, strat_diag_to, strat_rand)
    count3 += matrix_game_strategies(n, strat_diag_to, strat_next_to)
    count4 += matrix_game_strategies(n, strat_rand, strat_next_to)
    count5 += matrix_game_strategies(n, strat_zero_rc, strat_rand)
    count6 += matrix_game_strategies(n, strat_zero_rc, strat_next_to)
    count7 += matrix_game_strategies(n, strat_rand, strat_diagonals)
    count8 += matrix_game_strategies(n, strat_diag_to, strat_diagonals)
    count9 += matrix_game_strategies(n, strat_zero_rc, strat_diagonals)
print(f"The probability of winning with these two random strategies is {count1/sims}")
print(f"The probability of winning with diag_to strategy vs random strategies is {count2/sims}")
print(f"The probability of winning with diag_to strategy vs next_to strategies is {count3/sims}")
print(f"The probability of winning with random vs next_to strategies is {count4/sims}")
print(f"The probability of winning with zero r/c strategy vs random is {count5/sims}")
print(f"The probability of winning with zero r/c strategy vs next_to strategies is {count6/sims}")
print(f"The probability of winning with random strategy vs diagonal strategies is {count7/sims}")
print(f"The probability of winning with diag_to strategy vs diagonal strategies is {count8/sims}")
print(f"The probability of winning with zero r/c strategy vs diagonal strategies is {count9/sims}")

#plt.bar([1,3,5,7,9,11], [count1/sims, count2/sims, count3/sims, count4/sims, count5/sims, count6/sims], 0.8, 
#        tick_label = ['rand vs rand', 'next to vs rand', 'next to vs diag to', 'rand vs diag to', 'zero r/c vs rand', 'zero r/c vs diag to'])
#plt.title('n = 4')
#plt.show()

The probability of winning with these two random strategies is 0.547
The probability of winning with diag_to strategy vs random strategies is 0.59
The probability of winning with diag_to strategy vs next_to strategies is 0.597
The probability of winning with random vs next_to strategies is 0.58
The probability of winning with zero r/c strategy vs random is 0.737
The probability of winning with zero r/c strategy vs next_to strategies is 0.732
The probability of winning with random strategy vs diagonal strategies is 0.582
The probability of winning with diag_to strategy vs diagonal strategies is 0.581
The probability of winning with zero r/c strategy vs diagonal strategies is 0.726


In [46]:
sims = 1e4
n = 5
rng = np.random.default_rng()
count1 = 0
count2 = 0
count3 = 0
count4 = 0 
count5 = 0
count6 = 0
count7 = 0
count8 = 0
count9 = 0
for i in range(int(sims)):
    count1 += matrix_game_strategies(n, strat_rand, strat_rand)
    count2 += matrix_game_strategies(n, strat_diag_to, strat_rand)
    count3 += matrix_game_strategies(n, strat_diag_to, strat_next_to)
    count4 += matrix_game_strategies(n, strat_rand, strat_next_to)
    count5 += matrix_game_strategies(n, strat_zero_rc, strat_rand)
    count6 += matrix_game_strategies(n, strat_zero_rc, strat_next_to)
    count7 += matrix_game_strategies(n, strat_rand, strat_diagonals)
    count8 += matrix_game_strategies(n, strat_diag_to, strat_diagonals)
    count9 += matrix_game_strategies(n, strat_zero_rc, strat_diagonals)
print(f"The probability of winning with these two random strategies is {count1/sims}")
print(f"The probability of winning with diag_to strategy vs random strategies is {count2/sims}")
print(f"The probability of winning with diag_to strategy vs next_to strategies is {count3/sims}")
print(f"The probability of winning with random vs next_to strategies is {count4/sims}")
print(f"The probability of winning with zero r/c strategy vs random is {count5/sims}")
print(f"The probability of winning with zero r/c strategy vs next_to strategies is {count6/sims}")
print(f"The probability of winning with random strategy vs diagonal strategies is {count7/sims}")
print(f"The probability of winning with diag_to strategy vs diagonal strategies is {count8/sims}")
print(f"The probability of winning with zero r/c strategy vs diagonal strategies is {count9/sims}")

#plt.bar([1,3,5,7,9,11], [count1/sims, count2/sims, count3/sims, count4/sims, count5/sims, count6/sims], 0.8, 
#        tick_label = ['rand vs rand', 'next to vs rand', 'next to vs diag to', 'rand vs diag to', 'zero r/c vs rand', 'zero r/c vs diag to'])
#plt.title('n = 4')
#plt.show()

KeyboardInterrupt: 

In [42]:
sims = 1e4
n = 6
rng = np.random.default_rng()
count1 = 0
count2 = 0
count3 = 0
count4 = 0 
count5 = 0
count6 = 0
count7 = 0
count8 = 0
count9 = 0
for i in range(int(sims)):
    count1 += matrix_game_strategies(n, strat_rand, strat_rand)
    count2 += matrix_game_strategies(n, strat_diag_to, strat_rand)
    count3 += matrix_game_strategies(n, strat_diag_to, strat_next_to)
    count4 += matrix_game_strategies(n, strat_rand, strat_next_to)
    count5 += matrix_game_strategies(n, strat_zero_rc, strat_rand)
    count6 += matrix_game_strategies(n, strat_zero_rc, strat_next_to)
    count7 += matrix_game_strategies(n, strat_rand, strat_diagonals)
    count8 += matrix_game_strategies(n, strat_diag_to, strat_diagonals)
    count9 += matrix_game_strategies(n, strat_zero_rc, strat_diagonals)
print(f"The probability of winning with these two random strategies is {count1/sims}")
print(f"The probability of winning with diag_to strategy vs random strategies is {count2/sims}")
print(f"The probability of winning with diag_to strategy vs next_to strategies is {count3/sims}")
print(f"The probability of winning with random vs next_to strategies is {count4/sims}")
print(f"The probability of winning with zero r/c strategy vs random is {count5/sims}")
print(f"The probability of winning with zero r/c strategy vs next_to strategies is {count6/sims}")
print(f"The probability of winning with random strategy vs diagonal strategies is {count7/sims}")
print(f"The probability of winning with diag_to strategy vs diagonal strategies is {count8/sims}")
print(f"The probability of winning with zero r/c strategy vs diagonal strategies is {count9/sims}")

#plt.bar([1,3,5,7,9,11], [count1/sims, count2/sims, count3/sims, count4/sims, count5/sims, count6/sims], 0.8, 
#        tick_label = ['rand vs rand', 'next to vs rand', 'next to vs diag to', 'rand vs diag to', 'zero r/c vs rand', 'zero r/c vs diag to'])
#plt.title('n = 4')
#plt.show()

The probability of winning with these two random strategies is 0.539
The probability of winning with next_to strategy vs random strategies is 0.544
The probability of winning with next_to strategy vs diag_to strategies is 0.538
The probability of winning with random vs diag_to strategies is 0.532
The probability of winning with zero r/c strategy vs random is 0.65
The probability of winning with zero r/c strategy vs diag_to strategies is 0.647
The probability of winning with random strategy vs diagonal strategies is 0.503
The probability of winning with next_to strategy vs diagonal strategies is 0.562
The probability of winning with zero r/c strategy vs diagonal strategies is 0.655


In [49]:
sims = 1e4
n0 = [4,5,6,8,10]
rng = np.random.default_rng()
count = np.zeros((9,5))
ticks = ['rand vs rand', 'diag to vs rand', 'diag to vs next to', 'rand vs next to', 'zero r/c vs rand', 'zero r/c vs next to', 'rand vs diagonals', 'diag to vs diagonals', 'zero r/c vs diagonals']
for ind,n in enumerate(n0):
    for i in range(int(sims)):
        count[:,ind] += [matrix_game_strategies(n, strat_rand, strat_rand),
                    matrix_game_strategies(n, strat_diag_to, strat_rand),
                    matrix_game_strategies(n, strat_diag_to, strat_next_to), 
                    matrix_game_strategies(n, strat_rand, strat_next_to),
                    matrix_game_strategies(n, strat_zero_rc, strat_rand),
                    matrix_game_strategies(n, strat_zero_rc, strat_next_to),
                    matrix_game_strategies(n, strat_rand, strat_diagonals),
                    matrix_game_strategies(n, strat_diag_to, strat_diagonals),
                    matrix_game_strategies(n, strat_zero_rc, strat_diagonals)]
#print(f"The probability of winning with these two random strategies is {count1/sims}")
#print(f"The probability of winning with next_to strategy vs random strategies is {count2/sims}")
#print(f"The probability of winning with next_to strategy vs diag_to strategies is {count3/sims}")
#print(f"The probability of winning with random vs diag_to strategies is {count4/sims}")
#print(f"The probability of winning with zero r/c strategy vs random is {count5/sims}")
#print(f"The probability of winning with zero r/c strategy vs diag_to strategies is {count6/sims}")

# x_array would be [1,3,5,7,9,11,13,15,17]
#    plt.plot(count[:,ind]/sims, label = f'strategy probabilities for n = {n0}')
#plt.title('Strategy probabilities')
#plt.legend()
#plt.grid()
#plt.xticks(ticks)
#plt.show()

#Takes around 42 mins!!!!!!!!!


ConversionError: Failed to convert value(s) to axis units: ['rand vs rand', 'diag to vs rand', 'diag to vs next to', 'rand vs next to', 'zero r/c vs rand', 'zero r/c vs next to', 'rand vs diagonals', 'diag to vs diagonals', 'zero r/c vs diagonals']

In [52]:
for i in range(len(n0)):
    plt.plot(count[:,i]/sims, label = f'strategy probabilities for n = {n0[i]}')
plt.title('Strategy probabilities')
plt.legend()
plt.grid()
plt.xticks(np.arange(0,9),ticks)
plt.xlabel('Strategy')
plt.ylabel('Probability of player 0 winning')
plt.show()